# SQLSum
Given a table events with the following structure:
```
  create table events (
	event_type integer not null,
	value integer not null,
	time timestamp not null,
	unique(event_type, time)
  );
```

write an SQL query that, for each event_type that has been registered more than once, returns the difference between the latest (i.e. the most recent in terms of time) and the second latest value. The table should be ordered by event_type (in ascending order).

For example, given the following data:

| event_type | value | time |
| :------------ | :------------ | :------------ |
| 2 | 5 | 2015-05-09 12:42:00 |
| 4 | -42 | 2015-05-09 13:19:57 |
| 2 | 2 | 2015-05-09 14:48:30 |
| 2 | 7 | 2015-05-09 12:54:39 |
| 3 | 16 | 2015-05-09 13:19:57 |
| 3 | 20 | 2015-05-09 15:01:09 |

your query should return the following rowset:

| event_type | value |
| :------------ | :------------ |
| 2 | -5 |
| 3 | 4 |

For the event_type 2, the latest value is 2 and the second latest value is 7, so the difference between them is −5.

The names of the columns in the rowset don't matter, but their order does.


In [ ]:
-- Implement your solution here
WITH ranked_events AS(
    SELECT
    event_type,
    value,
    time,
    ROW_NUMBER() OVER(PARTITION BY event_type ORDER BY time DESC) AS rn
    FROM events
),
latest_values AS (
    SELECT
        event_type,
        MAX(CASE WHEN rn=1 THEN value END) AS latest_value,
        MAX(CASE WHEN rn = 2 THEN value END) AS second_latest_value
        FROM ranked_events
        WHERE rn <= 2
        GROUP BY event_type
)

SELECT
event_type,
latest_value - second_latest_value AS value_difference
FROM latest_values
WHERE second_latest_value IS NOT NULL
ORDER BY event_type;